In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from google.colab import files

# If datasets are in google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Load VISO dataset
# Using car 05 from NAS
# Images are a resolution of 1024x1024


IMG_FOLDER = "/content/gdrive/MyDrive/img" # Path to folder with images
GT_CSV = "/content/gdrive/MyDrive/gt_centroid.csv" # Path to csv file with ground truth labels
X_RES = 1024
Y_RES = 1024

IMG_COUNT = 0
for f in os.listdir(IMG_FOLDER):
  IMG_COUNT += 1

gt = pd.read_csv(GT_CSV, header=None)
gt.columns = ["frame", "id", "x", "y", "idk1", "idk2", "idk3", "idk4", "idk5", "idk6"]
# per documentation: ID,LATITUDE,LONGITUDE,X,Y,TIME,FRAME_NUMBER,TYPE
# The numbers don't make sense with these labels though, and there are more columns than given labels.
# ASSUMPTION:3 columns 3 and 4 are x and y, and labelled with (0, 0) at the top left

gt.head()

,frame,id,x,y,idk1,idk2,idk3,idk4,idk5,idk6
0,1,1,925.5,851.0,5,6,1,-1,-1,-1
1,1,2,905.5,787.5,5,7,1,-1,-1,-1
2,1,3,884.5,735.0,11,12,1,-1,-1,-1
3,1,4,868.5,711.0,7,6,1,-1,-1,-1
4,1,5,855.5,696.0,5,6,1,-1,-1,-1


In [ ]:
# KERAS
class ClusterNet(keras.layers.Layer):
  def __init__(self, size, conv_size=(3, 3), stride=2):
    super().__init__()
    self.k = self.add_weight(
        shape=conv_size, initializer="random_normal", trainable=True
    )
    # self.b = self.add_weight(shape=(size[1],), initializer="zeros", trainable=True)
    self.s = stride

  def call(self, input):
    final = []
    for sample in input:
      #print(sample.shape)
      h = sample.shape[0]
      w = sample.shape[1]
      kh = self.k.shape[0]
      kw = self.k.shape[1]
      out = np.zeros(((sample.shape[0] - kh)//self.s + 1, (sample.shape[1] - kw)//self.s + 1))
      #print(out.shape)
      x = 0
      y = 0
      for i in range(0, h - kh + 1, self.s):
        for j in range(0, w - kw + 1, self.s):
          curr = sample[i:i+kh, j:j+kw]
          # CURRENT ERRORS THROW AT THIS POINT
          # The calculation for out[y, x] do not work because Keras uses symbolic tensors,
          # which don't hold actual values themselves, and therefore cannot be used to
          # calculate and assign the final output value.
          # Main workaround worth trying may just be to move from the Keras library
          # to PyTorch to avoid symbolic tensors.
          out[y, x] = tf.convert_to_tensor(tf.reduce_sum(curr * self.k)) # + self.b
          x += 1
        y += 1
      final.append(out)
    return tf.convert_to_tensor(final)

In [ ]:
# Double check the output sizes after each ClusterNet layer.

model = tf.keras.models.Sequential([
  ClusterNet(size=(X_RES, Y_RES), conv_size=(3, 3), stride=2),
  tf.keras.layers.PReLU(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  ClusterNet(size=(511, 511), conv_size=(3, 3), stride=2),
  tf.keras.layers.PReLU(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  ClusterNet(size=(255, 255), conv_size=(3, 3), stride=1),
  tf.keras.layers.PReLU(),
  tf.keras.layers.BatchNormalization(),

  ClusterNet(size=(253, 253), conv_size=(1, 1), stride=1),
  tf.keras.layers.PReLU(),
  tf.keras.layers.BatchNormalization(),

  ClusterNet(size=(253, 253), conv_size=(3, 3), stride=1),
  tf.keras.layers.PReLU(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  ClusterNet(size=(251, 251), conv_size=(3, 3), stride=1),
  tf.keras.layers.PReLU(),
  tf.keras.layers.BatchNormalization(),

  ClusterNet(size=(249, 249), conv_size=(1, 1), stride=1),
  tf.keras.layers.PReLU(),
  tf.keras.layers.BatchNormalization()
])

In [ ]:
y = []
filter = gt[['frame', 'id', 'x', 'y']]
for i in range(1, gt['frame'].max() + 1):
  temp = filter[filter['frame'] == i]
  out = np.zeros((X_RES, Y_RES))
  for i, row in temp.iterrows():
    out[int(row.loc['y']), int(row.loc['x'])] = 255 #255 or 1?
  y.append(out)
y[0].sum()/255

63.0

In [ ]:
files = sorted(os.listdir(IMG_FOLDER))
X = []
for filename in files:
  img_path = IMG_FOLDER + "/" + filename
  X.append(cv2.imread(img_path, cv2.IMREAD_GRAYSCALE))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
with tf.device('/gpu:0'):
  X_train = tf.convert_to_tensor(X_train)
  X_test = tf.convert_to_tensor(X_test)
  y_train = tf.convert_to_tensor(y_train)
  y_test = tf.convert_to_tensor(y_test)

  X_train = tf.cast(X_train, tf.float32)
  X_test = tf.cast(X_test, tf.float32)
  y_train = tf.cast(y_train, tf.float32)
  y_test = tf.cast(y_test, tf.float32)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_test, y_test))

Epoch 1/10


AttributeError: ignored